In [2]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


/home/tg/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.data.shape)
print(boston.feature_names)

(506, 13)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [4]:
dfX = boston.data
dfY = np.array([boston.target]).transpose()
print(dfY.shape)
assert dfX.shape[0] == dfY.shape[0]
n = dfX.shape[0]


(506, 1)


In [5]:
test_split_rule = [i % 7 == 0 for i in range(n)]
train_split_rule = np.invert(test_split_rule)
testX, testY = dfX[test_split_rule], dfY[test_split_rule]
trainX, trainY = dfX[train_split_rule], dfY[train_split_rule]

/home/tg/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()


# 1. Histograms and pearson correlation



In [ ]:
n_attrs = trainX.shape[1]
attrs = [ trainX[:, i] for i in range(n_attrs)]
distribs = [(a.mean(), a.std())for a in attrs]

corrs = np.zeros(shape=(n_attrs, n_attrs))
for i, a in enumerate(attrs):
    for j in range(0, i + 1):
        b = attrs[j]
        # COV[A,B] = E[X - muX] E[Y - muY] = E[XY] - E[X]E[Y]
        # Pearson CORR[A, B] = COV[A,B] / (sigmaX * sigmaY)
        cov_ab = np.multiply(a, b).mean() - distribs[i][0] * distribs[j][0]
        cor_ab = cov_ab / (distribs[i][1] * distribs[j][1])
        corrs[i][j] = cor_ab

print("Correlations")
print(pd.DataFrame(corrs))
plt.imshow(corrs, cmap='hot', interpolation='nearest')
plt.show()

plt.hist(trainY, bins=bins)
plt.title("Histogram of Price with %d bins" % bins)
plt.show()

print("Histogram")
bins = 10
for i, attr in enumerate(attrs):
    plt.hist(attr, bins=bins)
    plt.title("Histogram of '%s' with %d bins" % (boston.feature_names[i], bins))
    plt.show()



# 2 Linear Regression

In [33]:



def predict(X, W):
    return np.matmul(X, W)

def MSECost(Y2, Y1):
    return float(np.sum((Y2 - Y1) ** 2) / len(Y2))

def gradient_desc(X, Y, W, alpha,
                  num_iter = 1000, conv_tol=0.01, print_interval = 100):
    c = float('inf')
    print("Learn Rate", alpha)
    for i in range(num_iter):
        predY = predict(X, W)
        diff = predY - Y
        delta = np.sum(np.multiply(X, diff), axis=0) # sum top to bottom for each attribute
        delta = np.array([delta]).transpose()        # restore vector shape of (n_attr x 1)
        
        W = (W - alpha * delta)
        if i % print_interval == 0:
            predY = predict(X, W)
            #print(np.concatenate((Y, predY), axis=1))
            newcost = MSECost(predY, Y)
            print("#%d, cost = %.8g" % (i, newcost))
            if np.isnan(newcost) or np.isinf(newcost) or np.isneginf(newcost):
                raise Exception("ERROR: number overflow, please adjust learning rate")
            diff = abs(newcost - c)
            c = newcost
            if diff < conv_tol:
                print("Converged with tolerance %f " % conv_tol)
                break            
        if i % (print_interval * 10) == 0:
            print(W.flatten())
    return W


# compute means and stds
class LinearRegression(object):
    
    def __init__(self, X, Y, learn_rate=0.001, num_iter=1000, conv_tol=0.1):
        
        self.means = X.mean(axis=0)
        self.stds = X.std(axis=0)        
        X = self.normalize(X)
        self.n_attrs = X.shape[1]
        W = np.random.rand(self.n_attrs, 1)
        self.W = gradient_desc(X, Y, W, alpha=learn_rate,
                                num_iter=num_iter, conv_tol=conv_tol)
    
    def normalize(self, X):
        X = (X - self.means) / self.stds
        # Bias is added as a weight to simplify the calculations
        X = np.insert(X, 0, 1, axis=1)
        return X
    
    def predict(self, X, normalize=True):
        if normalize:
            X = self.normalize(X)
        return np.matmul(X, self.W)
            
    
alpha = 0.0001
conv_tol = 0.000001
num_iter = 10000
linreg = LinearRegression(trainX, trainY, alpha, num_iter, conv_tol)


print("\n\n")
predY = linreg.predict(trainX)
train_mse_cost = MSECost(predY, trainY)
print('Train MSE::', train_mse_cost)


predY = linreg.predict(testX)
test_mse_cost = MSECost(predY, testY)
print('Test MSE::', test_mse_cost)

Learn Rate 0.0001
#0, cost = 538.50243
[ 1.10282596 -0.04495359  0.8220239   0.56757461  0.50598767  0.09673651
  1.11622142  0.63148602  0.46484325  0.58576546  0.16509781  0.12465829
  0.59073659  0.5383079 ]
#100, cost = 21.886264
#200, cost = 21.205122
#300, cost = 21.055035
#400, cost = 21.002407
#500, cost = 20.978718
#600, cost = 20.966388
#700, cost = 20.959521
#800, cost = 20.955589
#900, cost = 20.953313
#1000, cost = 20.95199
[ 22.46351039  -0.9603431    1.03157636  -0.21743147   0.92802198
  -1.61141017   2.73312084  -0.27506989  -3.11621861   2.3747743   -1.78702
  -1.88317389   0.82828193  -3.6714615 ]
#1100, cost = 20.95122
#1200, cost = 20.950771
#1300, cost = 20.95051
#1400, cost = 20.950357
#1500, cost = 20.950269
#1600, cost = 20.950217
#1700, cost = 20.950187
#1800, cost = 20.950169
#1900, cost = 20.950159
#2000, cost = 20.950153
[ 22.46351039  -0.96686743   1.04451163  -0.17646787   0.92183647
  -1.62007604   2.72620882  -0.26775038  -3.11274196   2.47632944
  -1.9

# Ridge Regression

In [ ]:
np.multiply(trainX[:, 1], trainX[:, 1])
